In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import wfdb
from tqdm.notebook import tqdm
import requests
from joblib import Parallel, delayed
from pathlib import Path
import wget
import time

In [2]:
MIMIC_DB_NAME = "mimic3wdb-matched"

all_records = requests.get(f"https://physionet.org/files/{MIMIC_DB_NAME}/1.0/RECORDS").text.split("/\n")
all_records = [record for record in all_records if len(record) > 0]

In [3]:
def is_good_measurement(record, measurement):
    header = requests.get(f"https://physionet.org/files/{MIMIC_DB_NAME}/1.0/{record}/{measurement}.hea").text
    return {
        "path": f"{record}/{measurement}",
        **{signal: signal in header for signal in ["ABP", "PLETH", "II", "V"]}
    }

def find_good_measurements(all_records, start, n_records=200):
    end = start + n_records
    current_records = all_records[start:end]
    good_measurements = []
    for record in tqdm(current_records, desc="Records"):
        record_measurements = requests.get(f"https://physionet.org/files/{MIMIC_DB_NAME}/1.0/{record}/RECORDS").text.split("\n")
        record_measurements = [measurement for measurement in record_measurements if len(measurement) > 0]
        results = Parallel(n_jobs=-1)(delayed(is_good_measurement)(record, measurement) for measurement in tqdm(record_measurements, desc="Record measurements"))
        results = [result for result in results if result is not None]
        good_measurements.extend(results)
        time.sleep(0.1)
    df = pd.DataFrame(good_measurements)
    df.to_csv(f"mimic_recordings/{start}-{end}.csv", index=False)
    return df

def download_measurements(measurements):
    for measurement_path in tqdm(measurements, desc="Downloading measurements"):
        directory, subject, recording = measurement_path.split("/")
        path = Path(f"mimic_db/{directory}/{subject}")
        path.mkdir(parents=True, exist_ok=True)
        url_dat = f'https://physionet.org/files/mimic3wdb-matched/1.0/{directory}/{subject}/{recording}.dat'
        url_hea = f'https://physionet.org/files/mimic3wdb-matched/1.0/{directory}/{subject}/{recording}.hea'
        wget.download(url_dat, out=str(path))
        wget.download(url_hea, out=str(path))

In [6]:
_ = find_good_measurements(all_records, start=8500, n_records=250)

Records:   0%|          | 0/250 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/83 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/23 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/20 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/10 [00:00<?, ?it/s]

...

Record measurements:   0%|          | 0/45 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/28 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/20 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/65 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/43 [00:00<?, ?it/s]

In [7]:
_ = find_good_measurements(all_records, start=8750, n_records=250)

Records:   0%|          | 0/250 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/488 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/127 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/21 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/16 [00:00<?, ?it/s]

...

Record measurements:   0%|          | 0/4 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/33 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/7 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/31 [00:00<?, ?it/s]

Record measurements:   0%|          | 0/25 [00:00<?, ?it/s]